In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-04-09 22:38:01,901 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [4]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield text
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield text
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield text
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield text
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield text
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield text
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield text
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield text
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [5]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-04-09 22:38:19,237 : INFO : loading Dictionary object from stem100_dictionary
2018-04-09 22:38:19,370 : INFO : loaded stem100_dictionary


In [6]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-04-09 22:38:19,402 : INFO : loaded corpus index from stem100.mm.index
2018-04-09 22:38:19,403 : INFO : initializing corpus reader from stem100.mm
2018-04-09 22:38:19,404 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [7]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


In [8]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [9]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield itemlist

In [10]:
%%time
model = gensim.models.Word2Vec(iter_token_corpus(), size=300, iter = 20, min_count=0, sample=.00001, workers=8)

2018-04-09 22:38:35,264 : INFO : collecting all words and their counts
2018-04-09 22:38:35,347 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-09 22:38:49,296 : INFO : PROGRESS: at sentence #10000, processed 11872635 words, keeping 28140 word types
2018-04-09 22:39:01,013 : INFO : PROGRESS: at sentence #20000, processed 22389436 words, keeping 31281 word types
2018-04-09 22:39:12,573 : INFO : PROGRESS: at sentence #30000, processed 32260628 words, keeping 32838 word types
2018-04-09 22:39:22,439 : INFO : PROGRESS: at sentence #40000, processed 40518216 words, keeping 34451 word types
2018-04-09 22:39:30,615 : INFO : PROGRESS: at sentence #50000, processed 47137590 words, keeping 35170 word types
2018-04-09 22:39:36,989 : INFO : PROGRESS: at sentence #60000, processed 53235450 words, keeping 36026 word types
2018-04-09 22:39:42,427 : INFO : PROGRESS: at sentence #70000, processed 58570547 words, keeping 36380 word types
2018-04-09 22:39:49,325 : INFO :

2018-04-09 22:43:45,535 : INFO : PROGRESS: at 0.29% examples, 181237 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:46,543 : INFO : PROGRESS: at 0.31% examples, 181081 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:47,544 : INFO : PROGRESS: at 0.32% examples, 180667 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:48,554 : INFO : PROGRESS: at 0.33% examples, 180453 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:49,561 : INFO : PROGRESS: at 0.33% examples, 179991 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:50,571 : INFO : PROGRESS: at 0.34% examples, 178964 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:51,588 : INFO : PROGRESS: at 0.35% examples, 179146 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:52,596 : INFO : PROGRESS: at 0.36% examples, 179923 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:53,606 : INFO : PROGRESS: at 0.38% examples, 180637 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:43:54,610 : INFO : PROGRESS: at 0.39% examples, 180974 words/s, in_q

2018-04-09 22:45:07,054 : INFO : PROGRESS: at 1.42% examples, 195745 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:08,066 : INFO : PROGRESS: at 1.43% examples, 195867 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:09,071 : INFO : PROGRESS: at 1.45% examples, 195946 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:10,075 : INFO : PROGRESS: at 1.46% examples, 196054 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:11,085 : INFO : PROGRESS: at 1.47% examples, 196221 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:12,085 : INFO : PROGRESS: at 1.48% examples, 196456 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:13,085 : INFO : PROGRESS: at 1.49% examples, 196647 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:14,094 : INFO : PROGRESS: at 1.50% examples, 196729 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:15,098 : INFO : PROGRESS: at 1.51% examples, 196726 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:45:16,112 : INFO : PROGRESS: at 1.52% examples, 196786 words/s, in_q

2018-04-09 22:46:28,767 : INFO : PROGRESS: at 2.66% examples, 200581 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:29,779 : INFO : PROGRESS: at 2.67% examples, 200483 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:30,782 : INFO : PROGRESS: at 2.68% examples, 200363 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:31,787 : INFO : PROGRESS: at 2.69% examples, 200259 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:32,789 : INFO : PROGRESS: at 2.69% examples, 200276 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:33,794 : INFO : PROGRESS: at 2.70% examples, 200378 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:34,801 : INFO : PROGRESS: at 2.71% examples, 200255 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:35,802 : INFO : PROGRESS: at 2.71% examples, 200182 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:36,806 : INFO : PROGRESS: at 2.72% examples, 200078 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:46:37,820 : INFO : PROGRESS: at 2.73% examples, 200031 words/s, in_q

2018-04-09 22:47:50,481 : INFO : PROGRESS: at 4.29% examples, 195574 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:51,487 : INFO : PROGRESS: at 4.32% examples, 195634 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:52,491 : INFO : PROGRESS: at 4.34% examples, 195680 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:53,509 : INFO : PROGRESS: at 4.36% examples, 195734 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:54,509 : INFO : PROGRESS: at 4.38% examples, 195808 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:55,523 : INFO : PROGRESS: at 4.40% examples, 195940 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:56,523 : INFO : PROGRESS: at 4.42% examples, 196146 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:57,533 : INFO : PROGRESS: at 4.43% examples, 196246 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:58,547 : INFO : PROGRESS: at 4.45% examples, 196323 words/s, in_qsize 0, out_qsize 0
2018-04-09 22:47:59,553 : INFO : PROGRESS: at 4.47% examples, 196365 words/s, in_q

...

2018-04-10 01:25:47,451 : INFO : PROGRESS: at 97.71% examples, 143680 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:48,459 : INFO : PROGRESS: at 97.71% examples, 143677 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:49,481 : INFO : PROGRESS: at 97.71% examples, 143673 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:50,491 : INFO : PROGRESS: at 97.72% examples, 143670 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:51,496 : INFO : PROGRESS: at 97.73% examples, 143667 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:52,517 : INFO : PROGRESS: at 97.73% examples, 143665 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:53,524 : INFO : PROGRESS: at 97.74% examples, 143662 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:54,550 : INFO : PROGRESS: at 97.74% examples, 143658 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:55,576 : INFO : PROGRESS: at 97.75% examples, 143655 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:25:56,583 : INFO : PROGRESS: at 97.75% examples, 143652 wor

2018-04-10 01:27:08,334 : INFO : PROGRESS: at 98.15% examples, 143484 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:09,336 : INFO : PROGRESS: at 98.16% examples, 143482 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:10,337 : INFO : PROGRESS: at 98.21% examples, 143479 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:11,343 : INFO : PROGRESS: at 98.28% examples, 143477 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:12,352 : INFO : PROGRESS: at 98.34% examples, 143476 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:13,372 : INFO : PROGRESS: at 98.41% examples, 143474 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:14,387 : INFO : PROGRESS: at 98.45% examples, 143471 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:15,401 : INFO : PROGRESS: at 98.49% examples, 143468 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:16,411 : INFO : PROGRESS: at 98.53% examples, 143464 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:27:17,426 : INFO : PROGRESS: at 98.55% examples, 143459 wor

2018-04-10 01:28:29,615 : INFO : PROGRESS: at 99.62% examples, 143288 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:30,621 : INFO : PROGRESS: at 99.62% examples, 143285 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:31,626 : INFO : PROGRESS: at 99.63% examples, 143282 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:32,638 : INFO : PROGRESS: at 99.63% examples, 143279 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:33,644 : INFO : PROGRESS: at 99.63% examples, 143275 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:34,662 : INFO : PROGRESS: at 99.63% examples, 143270 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:35,672 : INFO : PROGRESS: at 99.64% examples, 143268 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:36,683 : INFO : PROGRESS: at 99.64% examples, 143264 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:37,694 : INFO : PROGRESS: at 99.64% examples, 143262 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:28:38,706 : INFO : PROGRESS: at 99.65% examples, 143264 wor

2018-04-10 01:29:51,669 : INFO : PROGRESS: at 99.86% examples, 142958 words/s, in_qsize 0, out_qsize 1
2018-04-10 01:29:52,776 : INFO : PROGRESS: at 99.87% examples, 142951 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:53,786 : INFO : PROGRESS: at 99.87% examples, 142942 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:54,794 : INFO : PROGRESS: at 99.87% examples, 142936 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:55,820 : INFO : PROGRESS: at 99.87% examples, 142925 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:56,846 : INFO : PROGRESS: at 99.87% examples, 142920 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:57,859 : INFO : PROGRESS: at 99.88% examples, 142916 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:58,868 : INFO : PROGRESS: at 99.88% examples, 142908 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:29:59,888 : INFO : PROGRESS: at 99.88% examples, 142904 words/s, in_qsize 0, out_qsize 0
2018-04-10 01:30:00,917 : INFO : PROGRESS: at 99.89% examples, 142899 wor

CPU times: user 4h 2min 13s, sys: 3min 30s, total: 4h 5min 44s
Wall time: 2h 51min 53s


In [11]:
model.save('word2vecsubsam')

2018-04-10 01:39:42,569 : INFO : saving Word2Vec object under word2vecsubsam, separately None
2018-04-10 01:39:42,624 : INFO : not storing attribute syn0norm
2018-04-10 01:39:42,625 : INFO : storing np array 'syn0' to word2vecsubsam.wv.syn0.npy
2018-04-10 01:39:42,938 : INFO : storing np array 'syn1neg' to word2vecsubsam.syn1neg.npy
2018-04-10 01:39:43,049 : INFO : not storing attribute cum_table
2018-04-10 01:39:43,933 : INFO : saved word2vecsubsam


In [12]:
model = gensim.models.Word2Vec.load('word2vecsubsam')

2018-04-10 01:39:54,156 : INFO : loading Word2Vec object from word2vecsubsam
2018-04-10 01:39:54,390 : INFO : loading wv recursively from word2vecsubsam.wv.* with mmap=None
2018-04-10 01:39:54,393 : INFO : loading syn0 from word2vecsubsam.wv.syn0.npy with mmap=None
2018-04-10 01:39:54,677 : INFO : setting ignored attribute syn0norm to None
2018-04-10 01:39:54,678 : INFO : loading syn1neg from word2vecsubsam.syn1neg.npy with mmap=None
2018-04-10 01:39:54,746 : INFO : setting ignored attribute cum_table to None
2018-04-10 01:39:54,747 : INFO : loaded word2vecsubsam


In [105]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

IndentationError: unexpected indent (<ipython-input-105-b0cfacef251c>, line 2)

In [13]:
#preprocess a word
def T(word):
    if word not in en_stop:
        return ps.stem(word)

In [14]:
%%time

import math
from six import iteritems
from six.moves import xrange
idf = {}
for word, freq in dictionary.dfs.iteritems():
    idf[dictionary.id2token[word]] = math.log(dictionary.num_docs) - math.log(freq)


CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 50 ms


In [15]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [16]:
w2v = model.wv
def vec(document):
    return np.add.reduce([idf[i]*w2v[i] for i in document if i in dictionary.token2id])

In [17]:
%%time
doc_vec = []
for x in iter_token_corpus():
    x = vec(x)
    doc_vec.append(x)

CPU times: user 36min 25s, sys: 19.1 s, total: 36min 44s
Wall time: 38min 1s


In [18]:
from scipy.spatial.distance import cosine
def get_scores(query):
    scores = []
    i = 0
    query = vec(query)
    for x in doc_vec:
        if isinstance(x, np.ndarray):
            score = 1 - cosine(x, query)
        else:
            score = -1000
        scores.append(score)
    return scores

In [19]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = get_scores(query)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        

CPU times: user 1min 42s, sys: 196 ms, total: 1min 42s
Wall time: 1min 42s


In [20]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.59650022538319936, 2852],
  ['ConsumerCourt_DCDRC_106530', '0', 0.36398613919424672, 31401],
  ['ConsumerCourt_DCDRC_107608', '1', 0.62070325527806125, 1939],
  ['ConsumerCourt_DCDRC_114291', '1', 0.69398418080678148, 425],
  ['ConsumerCourt_DCDRC_114382', '1', 0.68863895143265863, 479],
  ['ConsumerCourt_DCDRC_118185', '1', 0.71434196576521458, 250],
  ['ConsumerCourt_DCDRC_130318', '0', 0.64129177926996916, 1348],
  ['ConsumerCourt_DCDRC_130570', '1', 0.64654658132804355, 1224],
  ['ConsumerCourt_DCDRC_131146', '1', 0.75375002164049143, 74],
  ['ConsumerCourt_DCDRC_131717', '1', 0.74291120822183976, 102],
  ['ConsumerCourt_DCDRC_131741', '1', 0.70750584353422752, 306],
  ['ConsumerCourt_DCDRC_131818', '1', 0.72801316237546143, 172],
  ['ConsumerCourt_DCDRC_131950', '0', 0.68440827303587459, 558],
  ['ConsumerCourt_DCDRC_131972', '0', 0.76028228623807936, 54],
  ['ConsumerCourt_DCDRC_132932', '0', 0.7950539906566314, 7],
  ['ConsumerCourt_D

In [23]:
model_name = 'w2v_idf_subsam'
# Store results in 1st sheet
wb = Workbook()
dest_filename = model_name + '.xlsx'
ws1 = wb.active
ws1.title = " "
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)


#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [28]:
x = f.parse(1)

In [36]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
f2 = pd.ExcelFile('all_results.xlsx')
append_df_to_excel("all_results.xlsx", f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel("all_results.xlsx", f.parse(2), sheet_name= f2.sheet_names[1])